In [4]:
# data

# q1
!which python
import pandas as pd
data = pd.read_excel('./AirQuality.xlsx')
data.describe()
data.fillna(data.mean())

df = data[['CO(GT)','NOx(GT)']].copy()

df[df['NOx(GT)'] < 0] = df['NOx(GT)'].mean()
df[df['CO(GT)'] < 0] = df['CO(GT)'].mean()
df[df['CO(GT)'] > 20] = df['CO(GT)'].mean()

print(df.describe())

/usr/local/bin/python
            CO(GT)      NOx(GT)
count  9357.000000  9357.000000
mean      8.945478   205.344185
std      12.610694   211.254046
min       0.100000     2.000000
25%       1.400000    34.000000
50%       2.400000   136.000000
75%       5.500000   281.500000
max      32.293720  1479.000000


In [5]:
# plotting CO and NO scatterplot to visualize the spread of data and if there can be a linear relationship
ax2 = df.plot.scatter(y='CO(GT)',x='NOx(GT)', colormap='viridis')


TypeError: You must first set_array for mappable

In [131]:
#using Sklearn to establish relationship

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import f_regression
import math

y = df[['CO(GT)']]
x = df[['NOx(GT)']]


reg = LinearRegression().fit(x,y)
print('Regression with data clean-up')
print ('score= ',reg.score(x,y))
print ('slope= ',reg.coef_)
print ('intercept= ',reg.intercept_)
p = reg.predict(x)
print( 'rms= ',math.sqrt(mean_squared_error(y, p)))
r2 = r2_score(y,p)

print('r2 value:', r2)

f = f_regression(x, y, center=True)

print ('f-stats: ',f[0])
print ('p-value: ',f[1])

# p-value for the coefficient being less than 0.05 can make us reject NULL hypothesis and we can confirm
# that there is relationship


Regression with data clean-up
score=  0.1333275107427686
slope=  [[-0.02179685]]
intercept=  [13.42133522]
rms=  11.739329008351651
r2 value: 0.1333275107427686
f-stats:  [1439.15825004]
p-value:  [4.68230193e-293]
CPU times: user 32.1 ms, sys: 4.71 ms, total: 36.8 ms
Wall time: 27.3 ms


/Users/pramanikpramanik/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [128]:
%%time
#using statsmodel to find the p-value and f stats to establish relationship


import numpy as np
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())
print ('p value(Prob (F-statistic)) for the estimator is almost zero as per the below summary')

                            OLS Regression Results                            
Dep. Variable:                 CO(GT)   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     1439.
Date:                Sat, 14 Dec 2019   Prob (F-statistic):          4.68e-293
Time:                        22:59:52   Log-Likelihood:                -36323.
No. Observations:                9357   AIC:                         7.265e+04
Df Residuals:                    9355   BIC:                         7.266e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.4213      0.169     79.291      0.0

/Users/pramanikpramanik/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [129]:
%%time
# using numpy functions for establishing relationship


import numpy as np


b=df.loc[:,'CO(GT)'].values#.reshape(-1,1)
a=df.loc[:,'NOx(GT)'].values#.reshape(-1,1)


A = np.vstack([a, np.ones(len(a))]).T

m, c = np.linalg.lstsq(A, b, rcond=None)[0]
residual = np.linalg.lstsq(A, b, rcond=None)[1]

print ('slope: ',m)
print ('intercept: ',c)
print (math.sqrt(residual[0]))

slope:  -0.02179685454344522
intercept:  13.421335218814898
1135.5639299326615
CPU times: user 1.98 ms, sys: 761 µs, total: 2.74 ms
Wall time: 1.93 ms


In [130]:
# statsmodel - 26 ms
# sklearn - 28 ms
# numpy - 1.93 - but P-values were not found, only best fit line was calculated.